In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
!pip install kafka-python

Defaulting to user installation because normal site-packages is not writeable
  Using cached kafka_python-2.0.2-py2.py3-none-any.whl (246 kB)



[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
!pip install pyspark

Defaulting to user installation because normal site-packages is not writeable
  Using cached pyspark-3.5.3.tar.gz (317.3 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached py4j-0.10.9.7-py2.py3-none-any.whl (200 kB)
  Running setup.py install for pyspark: started
  Running setup.py install for pyspark: still running...
  Running setup.py install for pyspark: finished with status 'done'


  DEPRECATION: pyspark is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [14]:
from kafka import KafkaProducer
from pyspark.sql import SparkSession
from pyspark.sql.types import DoubleType
from pyspark.sql.functions import rand, randn, mean, stddev, to_date, col, when, dayofyear
from pyspark.sql.functions import lit, sum as _sum, udf, hour, collect_list, struct, from_json
from datetime import datetime, timedelta
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
import json
import random
import time
import csv
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml import Pipeline
from pyspark.sql.types import StructType, StructField, StringType, DoubleType
from pyspark.sql.window import Window

In [17]:
!javac -version

In [18]:
app_name = "IoT_Home_Appliances"
spark = SparkSession.builder.appName(app_name).getOrCreate()
spark

Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: java.lang.RuntimeException: java.io.FileNotFoundException: java.io.FileNotFoundException: HADOOP_HOME and hadoop.home.dir are unset. -see https://wiki.apache.org/hadoop/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:735)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:270)
	at org.apache.hadoop.fs.FileUtil.chmod(FileUtil.java:1139)
	at org.apache.hadoop.fs.FileUtil.chmod(FileUtil.java:1125)
	at org.apache.spark.util.Utils$.fetchFile(Utils.scala:489)
	at org.apache.spark.SparkContext.addFile(SparkContext.scala:1790)
	at org.apache.spark.SparkContext.$anonfun$new$16(SparkContext.scala:528)
	at org.apache.spark.SparkContext.$anonfun$new$16$adapted(SparkContext.scala:528)
	at scala.collection.immutable.List.foreach(List.scala:431)
	at org.apache.spark.SparkContext.<init>(SparkContext.scala:528)
	at org.apache.spark.api.java.JavaSparkC

# Part 1 -> Day wise cost consumption on devices

# Data Generation

## Using the pre existing data for 4 months for Delhi city

In [6]:
# df = spark.read.csv('/kaggle/input/daily-climate-time-series-data/DailyDelhiClimateTrain.csv', header=True, inferSchema=True)
# df.show()

+----------+------------------+------------------+------------------+------------------+
|      date|          meantemp|          humidity|        wind_speed|      meanpressure|
+----------+------------------+------------------+------------------+------------------+
|2013-01-01|              10.0|              84.5|               0.0|1015.6666666666666|
|2013-01-02|               7.4|              92.0|              2.98|            1017.8|
|2013-01-03| 7.166666666666667|              87.0| 4.633333333333334|1018.6666666666666|
|2013-01-04| 8.666666666666666| 71.33333333333333|1.2333333333333334|1017.1666666666666|
|2013-01-05|               6.0| 86.83333333333333|3.6999999999999997|            1016.5|
|2013-01-06|               7.0|              82.8|              1.48|            1018.0|
|2013-01-07|               7.0|              78.6|               6.3|            1020.0|
|2013-01-08| 8.857142857142858|63.714285714285715| 7.142857142857143|1018.7142857142857|
|2013-01-09|         

In [ ]:

# Initialize Kafka producer
producer = KafkaProducer(
    bootstrap_servers='localhost:9092',
    value_serializer=lambda v: json.dumps(v).encode('utf-8')
)

# Path to your CSV file
file_path = 'C:\Users\hp\OneDrive\Desktop\IoT_DA331_Project\DailyDelhiClimateTrain.csv'

# Function to send data to Kafka
def send_data_to_kafka(file_path, topic):
    with open(file_path, 'r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            producer.send(topic, row)
            time.sleep(1)  # Simulate real-time data feed

# Call the function
send_data_to_kafka(file_path, 'raw-climate-data')

In [ ]:
# Define the schema for the incoming data
schema = StructType([
    StructField("date", StringType()),
    StructField("humidity", DoubleType()),
    StructField("meantemp", DoubleType()),
    StructField("wind_speed", DoubleType()),
    StructField("meanpressure", DoubleType())
])

# Read data from Kafka
df = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "raw-climate-data") \
    .option("startingOffsets", "earliest") \
    .load() \
    .selectExpr("CAST(value AS STRING) as json")

df = df.select(from_json(col("json"), schema).alias("data")).select("data.*")
df = df.withColumn('date', to_date(df.date, 'yyyy-MM-dd'))
df

## Keeping only the temperature and humidity columns

In [7]:
df = df.drop("meanpressure","wind_speed")
df.show()

+----------+------------------+------------------+
|      date|          meantemp|          humidity|
+----------+------------------+------------------+
|2013-01-01|              10.0|              84.5|
|2013-01-02|               7.4|              92.0|
|2013-01-03| 7.166666666666667|              87.0|
|2013-01-04| 8.666666666666666| 71.33333333333333|
|2013-01-05|               6.0| 86.83333333333333|
|2013-01-06|               7.0|              82.8|
|2013-01-07|               7.0|              78.6|
|2013-01-08| 8.857142857142858|63.714285714285715|
|2013-01-09|              14.0|             51.25|
|2013-01-10|              11.0|              62.0|
|2013-01-11|15.714285714285714|51.285714285714285|
|2013-01-12|              14.0|              74.0|
|2013-01-13|15.833333333333334| 75.16666666666667|
|2013-01-14|12.833333333333334| 88.16666666666667|
|2013-01-15|14.714285714285714| 71.85714285714286|
|2013-01-16|13.833333333333334| 86.66666666666667|
|2013-01-17|              16.5|

In [8]:
df.count()

1462

In [10]:
stats = df.select(
    mean(df.humidity).alias("mean_humidity"),
    stddev(df.humidity).alias("stddev_humidity")
).collect()

mean_humidity = stats[0]['mean_humidity']
stddev_humidity = stats[0]['stddev_humidity']

In [11]:
mean_humidity, stddev_humidity

(60.77170158004638, 16.769652268485313)

## Linear regression on the given data to generate more data -> year 2017 -> 1 to 5 months data generated using linear regression

In [12]:
# Convert the 'date' column to actual date type (assuming date format is 'YYYY-MM-DD')
df = df.withColumn('date', to_date(df.date, 'yyyy-MM-dd'))

# Prepare features using VectorAssembler
assembler = VectorAssembler(inputCols=["humidity"], outputCol="features")
df_assembled = assembler.transform(df)

# Assume 'meantemp' is the label column
df_assembled = df_assembled.select('date', 'features', 'meantemp')

In [13]:
# Split the data into training and testing (assuming you use all data for training here)
train_data = df_assembled  # As this is a simulation, we'll assume all the data is for training

# Apply Linear Regression
linear_reg = LinearRegression(featuresCol='features', labelCol='meantemp')
model = linear_reg.fit(train_data)


24/10/26 09:47:23 WARN Instrumentation: [7033035a] regParam is zero, which might cause numerical instability and overfitting.
24/10/26 09:47:23 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
24/10/26 09:47:24 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [14]:
df_assembled.show()

+----------+--------------------+------------------+
|      date|            features|          meantemp|
+----------+--------------------+------------------+
|2013-01-01|              [84.5]|              10.0|
|2013-01-02|              [92.0]|               7.4|
|2013-01-03|              [87.0]| 7.166666666666667|
|2013-01-04| [71.33333333333333]| 8.666666666666666|
|2013-01-05| [86.83333333333333]|               6.0|
|2013-01-06|              [82.8]|               7.0|
|2013-01-07|              [78.6]|               7.0|
|2013-01-08|[63.714285714285715]| 8.857142857142858|
|2013-01-09|             [51.25]|              14.0|
|2013-01-10|              [62.0]|              11.0|
|2013-01-11|[51.285714285714285]|15.714285714285714|
|2013-01-12|              [74.0]|              14.0|
|2013-01-13| [75.16666666666667]|15.833333333333334|
|2013-01-14| [88.16666666666667]|12.833333333333334|
|2013-01-15| [71.85714285714286]|14.714285714285714|
|2013-01-16| [86.66666666666667]|13.8333333333

In [15]:
start_date = "2017-01-02"
end_date = "2017-05-31"
date_index = pd.date_range(start_date, end_date, freq='D')

num_days = len(date_index)
random_humidities = np.random.normal(loc=mean_humidity, scale=stddev_humidity, size=num_days)

# Ensure humidity values are within a realistic range (e.g., 0 to 100)
random_humidities = np.clip(random_humidities, 0, 100)

# Create a Pandas DataFrame with the date range and generated humidity values
pandas_df = pd.DataFrame({
    'date': date_index,
    'humidity': random_humidities
})

pandas_df

,date,humidity
0,2017-01-02,71.496819
1,2017-01-03,65.403526
2,2017-01-04,72.558460
3,2017-01-05,84.753457
4,2017-01-06,62.499203
...,...,...
145,2017-05-27,62.007060
146,2017-05-28,76.851348
147,2017-05-29,84.740421
148,2017-05-30,65.595326


In [16]:
new_data = spark.createDataFrame(pandas_df)

# Convert 'date' to the proper date type
new_data = new_data.withColumn('date', to_date(new_data.date))

# Prepare the features using VectorAssembler
assembler = VectorAssembler(inputCols=["humidity"], outputCol="features")
new_data = assembler.transform(new_data)

In [17]:
# Predict using the model
predicted_data = model.transform(new_data)
generated_data = predicted_data.select('date', 'humidity', 'prediction')
generated_data.show()

+----------+------------------+------------------+
|      date|          humidity|        prediction|
+----------+------------------+------------------+
|2017-01-02| 71.49681930574594|22.807628515018006|
|2017-01-03| 65.40352629976692|24.334708664709478|
|2017-01-04| 72.55846026786399|22.541563706094653|
|2017-01-05| 84.75345728030592|19.485295470680178|
|2017-01-06|62.499203277320014|25.062580118046554|
|2017-01-07| 80.88326178331549|20.455230563753847|
|2017-01-08|45.360867721688265| 29.35773086823631|
|2017-01-09| 52.06455553454433|27.677675670054306|
|2017-01-10|30.638633297528077| 33.04736665732097|
|2017-01-11| 60.64035290496075|25.528438809248676|
|2017-01-12| 54.02992937306196|27.185120433899044|
|2017-01-13| 65.40724900503415|24.333775693110844|
|2017-01-14| 48.62223693962528|28.540377724601264|
|2017-01-15| 66.64858369250346|24.022676660371438|
|2017-01-16| 44.20369687375285| 29.64773704879416|
|2017-01-17| 59.23294274177648|25.881159106888035|
|2017-01-18| 66.29425153005164|

In [18]:
generated_data = generated_data.withColumnRenamed("prediction","meantemp")

In [19]:
generated_data.show()

+----------+------------------+------------------+
|      date|          humidity|          meantemp|
+----------+------------------+------------------+
|2017-01-02| 71.49681930574594|22.807628515018006|
|2017-01-03| 65.40352629976692|24.334708664709478|
|2017-01-04| 72.55846026786399|22.541563706094653|
|2017-01-05| 84.75345728030592|19.485295470680178|
|2017-01-06|62.499203277320014|25.062580118046554|
|2017-01-07| 80.88326178331549|20.455230563753847|
|2017-01-08|45.360867721688265| 29.35773086823631|
|2017-01-09| 52.06455553454433|27.677675670054306|
|2017-01-10|30.638633297528077| 33.04736665732097|
|2017-01-11| 60.64035290496075|25.528438809248676|
|2017-01-12| 54.02992937306196|27.185120433899044|
|2017-01-13| 65.40724900503415|24.333775693110844|
|2017-01-14| 48.62223693962528|28.540377724601264|
|2017-01-15| 66.64858369250346|24.022676660371438|
|2017-01-16| 44.20369687375285| 29.64773704879416|
|2017-01-17| 59.23294274177648|25.881159106888035|
|2017-01-18| 66.29425153005164|

In [20]:
final_df = df.unionByName(generated_data)
final_df.show()

+----------+------------------+------------------+
|      date|          meantemp|          humidity|
+----------+------------------+------------------+
|2013-01-01|              10.0|              84.5|
|2013-01-02|               7.4|              92.0|
|2013-01-03| 7.166666666666667|              87.0|
|2013-01-04| 8.666666666666666| 71.33333333333333|
|2013-01-05|               6.0| 86.83333333333333|
|2013-01-06|               7.0|              82.8|
|2013-01-07|               7.0|              78.6|
|2013-01-08| 8.857142857142858|63.714285714285715|
|2013-01-09|              14.0|             51.25|
|2013-01-10|              11.0|              62.0|
|2013-01-11|15.714285714285714|51.285714285714285|
|2013-01-12|              14.0|              74.0|
|2013-01-13|15.833333333333334| 75.16666666666667|
|2013-01-14|12.833333333333334| 88.16666666666667|
|2013-01-15|14.714285714285714| 71.85714285714286|
|2013-01-16|13.833333333333334| 86.66666666666667|
|2013-01-17|              16.5|

## Introducing the price column for prediction purposes

* The price increases with temperature, assuming higher temperatures lead to higher demand for cooling appliances
* The price decreases with humidity, assuming higher humidity reduces the efficiency of cooling, thus affecting prices.


* Price increases by 1 dollar for every degree Celsius above a baseline temperature of 20°C
* Price decreases by 0.5 dollar for every percentage point of humidity above a baseline of 50%.

In [21]:
baseline_temp = 20.0
baseline_humidity = 50.0

In [22]:
final_df = final_df.withColumn("price", 
                   when(col("meantemp") > baseline_temp, (col("meantemp") - baseline_temp) * 1) \
                   .otherwise(0) - \
                   when(col("humidity") > baseline_humidity, (col("humidity") - baseline_humidity) * 0.5) \
                   .otherwise(0)
                  )

# Adjust price to have a base price of $50
final_df = final_df.withColumn("price", col("price") + 50)

final_df.show()

+----------+------------------+------------------+------------------+
|      date|          meantemp|          humidity|             price|
+----------+------------------+------------------+------------------+
|2013-01-01|              10.0|              84.5|             32.75|
|2013-01-02|               7.4|              92.0|              29.0|
|2013-01-03| 7.166666666666667|              87.0|              31.5|
|2013-01-04| 8.666666666666666| 71.33333333333333|39.333333333333336|
|2013-01-05|               6.0| 86.83333333333333|31.583333333333336|
|2013-01-06|               7.0|              82.8|              33.6|
|2013-01-07|               7.0|              78.6|              35.7|
|2013-01-08| 8.857142857142858|63.714285714285715| 43.14285714285714|
|2013-01-09|              14.0|             51.25|            49.375|
|2013-01-10|              11.0|              62.0|              44.0|
|2013-01-11|15.714285714285714|51.285714285714285| 49.35714285714286|
|2013-01-12|        

# Data Visualization -> Time series plot for the above data

In [23]:
pd_df = final_df.toPandas()
pd_df

,date,meantemp,humidity,price
0,2013-01-01,10.000000,84.500000,32.750000
1,2013-01-02,7.400000,92.000000,29.000000
2,2013-01-03,7.166667,87.000000,31.500000
3,2013-01-04,8.666667,71.333333,39.333333
4,2013-01-05,6.000000,86.833333,31.583333
...,...,...,...,...
1607,2017-05-27,25.185919,62.007060,49.182390
1608,2017-05-28,21.465695,76.851348,38.040021
1609,2017-05-29,19.488563,84.740421,32.629790
1610,2017-05-30,24.286641,65.595326,46.488978


In [24]:
pd_df['date'] = pd.to_datetime(pd_df['date'])

In [25]:
# Create a figure with subplots
fig = make_subplots(rows=3, cols=1,
                    shared_xaxes=True,
                    vertical_spacing=0.02,  # Vertical spacing between plots
                    subplot_titles=("Mean Temperature", "Humidity", "Price"))

# Temperature Plot
fig.add_trace(
    go.Scatter(x=pd_df['date'], y=pd_df['meantemp'], name='Mean Temperature', mode='lines', line=dict(color='red')),
    row=1, col=1
)

# Humidity Plot
fig.add_trace(
    go.Scatter(x=pd_df['date'], y=pd_df['humidity'], name='Humidity', mode='lines', line=dict(color='blue')),
    row=2, col=1
)

# Price Plot
fig.add_trace(
    go.Scatter(x=pd_df['date'], y=pd_df['price'], name='Price', mode='lines', line=dict(color='green')),
    row=3, col=1
)

# Update x-axis and y-axis labels
fig.update_xaxes(title_text="Date", row=3, col=1)
fig.update_yaxes(title_text="Temperature (°C)", row=1, col=1)
fig.update_yaxes(title_text="Humidity (%)", row=2, col=1)
fig.update_yaxes(title_text="Price ($)", row=3, col=1)

# Update layout and plot size
fig.update_layout(height=900, width=700, title_text="Data Over Time", showlegend=False)

# Show plot
fig.show()

In [26]:
final_df.show()

+----------+------------------+------------------+------------------+
|      date|          meantemp|          humidity|             price|
+----------+------------------+------------------+------------------+
|2013-01-01|              10.0|              84.5|             32.75|
|2013-01-02|               7.4|              92.0|              29.0|
|2013-01-03| 7.166666666666667|              87.0|              31.5|
|2013-01-04| 8.666666666666666| 71.33333333333333|39.333333333333336|
|2013-01-05|               6.0| 86.83333333333333|31.583333333333336|
|2013-01-06|               7.0|              82.8|              33.6|
|2013-01-07|               7.0|              78.6|              35.7|
|2013-01-08| 8.857142857142858|63.714285714285715| 43.14285714285714|
|2013-01-09|              14.0|             51.25|            49.375|
|2013-01-10|              11.0|              62.0|              44.0|
|2013-01-11|15.714285714285714|51.285714285714285| 49.35714285714286|
|2013-01-12|        

In [27]:
pd_final_df = final_df.toPandas()
pd_final_df

,date,meantemp,humidity,price
0,2013-01-01,10.000000,84.500000,32.750000
1,2013-01-02,7.400000,92.000000,29.000000
2,2013-01-03,7.166667,87.000000,31.500000
3,2013-01-04,8.666667,71.333333,39.333333
4,2013-01-05,6.000000,86.833333,31.583333
...,...,...,...,...
1607,2017-05-27,25.185919,62.007060,49.182390
1608,2017-05-28,21.465695,76.851348,38.040021
1609,2017-05-29,19.488563,84.740421,32.629790
1610,2017-05-30,24.286641,65.595326,46.488978


In [28]:
pandas_df.to_csv('/kaggle/working/final_pd_data.csv', index=False)


# Prediction

Predicting the price of test data by training the model on the features and price of train data

In [29]:
# First, ensure 'date' is converted to a numeric feature like day of year
final_df = final_df.withColumn("day_of_year", dayofyear(col("date")))

# Prepare features and target. Update 'required_features' to include 'day_of_year' instead of 'date'
required_features = ['meantemp', 'humidity', 'day_of_year']
assembler = VectorAssembler(inputCols=required_features, outputCol="features")

# Split data
train_data, test_data = final_df.randomSplit([0.8, 0.2], seed=42)

# Train model
rf = RandomForestRegressor(featuresCol="features", labelCol="price", numTrees=100)
pipeline = Pipeline(stages=[assembler, rf])

model = pipeline.fit(train_data)

# Make predictions
predictions = model.transform(test_data)

# Select example rows to display
predictions.select("prediction", "price", "features").show(5)

# Evaluate the model using the RMSE metric
evaluator = RegressionEvaluator(labelCol="price", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data =", rmse)


+-----------------+------------------+--------------------+
|       prediction|             price|            features|
+-----------------+------------------+--------------------+
|32.43788295051903|              31.5|[7.16666666666666...|
|35.30710830435685|              35.7|      [7.0,78.6,7.0]|
| 45.8291735683571|            49.375|    [14.0,51.25,9.0]|
|32.75557491511467|30.916666666666664|[12.8333333333333...|
| 36.8076615498987|37.357142857142854|[11.2857142857142...|
+-----------------+------------------+--------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) on test data = 1.518886542262354


# Control logic on the use of appliances

In [30]:
def control_logic(predicted_price, threshold=35.0):
    if predicted_price > threshold:
        return "Turn Off"
    else:
        return "Turn On"

# Register the UDF
control_logic_udf = udf(control_logic, StringType())

In [31]:
control_logic_udf

<function __main__.control_logic(predicted_price, threshold=35.0)>

In [32]:
# Assuming 'predictions' DataFrame has a column 'prediction' which contains predicted prices
predictions = predictions.withColumn("action", control_logic_udf(predictions["prediction"]))

In [33]:
predictions.show()

+----------+------------------+------------------+------------------+-----------+--------------------+------------------+--------+
|      date|          meantemp|          humidity|             price|day_of_year|            features|        prediction|  action|
+----------+------------------+------------------+------------------+-----------+--------------------+------------------+--------+
|2013-01-03| 7.166666666666667|              87.0|              31.5|          3|[7.16666666666666...| 32.43788295051903| Turn On|
|2013-01-07|               7.0|              78.6|              35.7|          7|      [7.0,78.6,7.0]| 35.30710830435685|Turn Off|
|2013-01-09|              14.0|             51.25|            49.375|          9|    [14.0,51.25,9.0]|  45.8291735683571|Turn Off|
|2013-01-14|12.833333333333334| 88.16666666666667|30.916666666666664|         14|[12.8333333333333...| 32.75557491511467| Turn On|
|2013-01-20|11.285714285714286| 75.28571428571429|37.357142857142854|         20|[1

In [34]:
predictions.select("prediction", "action").show()

+------------------+--------+
|        prediction|  action|
+------------------+--------+
| 32.43788295051903| Turn On|
| 35.30710830435685|Turn Off|
|  45.8291735683571|Turn Off|
| 32.75557491511467| Turn On|
|  36.8076615498987|Turn Off|
| 44.23200678571799|Turn Off|
| 39.50081880343035|Turn Off|
| 34.33335042944989| Turn On|
| 42.14817948141224|Turn Off|
| 33.91116394034727| Turn On|
| 33.91116394034727| Turn On|
| 35.82995701410525|Turn Off|
| 41.89784356386328|Turn Off|
| 39.40884130049617|Turn Off|
| 40.18576533585722|Turn Off|
|46.570131724983575|Turn Off|
| 41.74547935158837|Turn Off|
|59.792241968189366|Turn Off|
|59.322609890946495|Turn Off|
| 60.56452475187566|Turn Off|
+------------------+--------+
only showing top 20 rows



# Plot

In [35]:
data = predictions.toPandas()
data['date'] = pd.to_datetime(data['date'])  # ensure 'date' is datetime type
data

,date,meantemp,humidity,price,day_of_year,features,prediction,action
0,2013-01-03,7.166667,87.000000,31.500000,3,"[7.166666666666667, 87.0, 3.0]",32.437883,Turn On
1,2013-01-07,7.000000,78.600000,35.700000,7,"[7.0, 78.6, 7.0]",35.307108,Turn Off
2,2013-01-09,14.000000,51.250000,49.375000,9,"[14.0, 51.25, 9.0]",45.829174,Turn Off
3,2013-01-14,12.833333,88.166667,30.916667,14,"[12.833333333333334, 88.16666666666667, 14.0]",32.755575,Turn On
4,2013-01-20,11.285714,75.285714,37.357143,20,"[11.285714285714286, 75.28571428571429, 20.0]",36.807662,Turn Off
...,...,...,...,...,...,...,...,...
278,2017-05-10,29.118410,46.315798,59.118410,130,"[29.118409644172576, 46.31579752092934, 130.0]",59.491092,Turn Off
279,2017-05-21,28.353395,49.368329,58.353395,141,"[28.353394613785973, 49.36832933143405, 141.0]",58.470676,Turn Off
280,2017-05-25,25.813689,59.502158,51.062610,145,"[25.81368935016623, 59.50215781659173, 145.0]",51.495052,Turn Off
281,2017-05-27,25.185919,62.007060,49.182390,147,"[25.1859194784635, 62.007059585751286, 147.0]",50.329194,Turn Off


In [36]:
# Create a subplot with 3 rows
fig = make_subplots(
    rows=3, cols=1,
    shared_xaxes=True,
    vertical_spacing=0.1,
    subplot_titles=("Mean Temperature (°C)", "Humidity (%)", "Price ($) with Action Indicators")
)

# Mean Temperature Trace
fig.add_trace(
    go.Scatter(x=data['date'], y=data['meantemp'], name='Mean Temperature', marker=dict(color='red')),
    row=1, col=1
)

# Humidity Trace
fig.add_trace(
    go.Scatter(x=data['date'], y=data['humidity'], name='Humidity', marker=dict(color='blue')),
    row=2, col=1
)

# Price Trace with Actions
colors = [ 'green' if action == 'Turn On' else 'red' for action in data['action']]
fig.add_trace(
    go.Scatter(x=data['date'], y=data['price'], name='Price',
               mode='lines+markers', marker=dict(color=colors, size=5)),
    row=3, col=1
)

# Update x-axis and y-axis labels
fig.update_xaxes(title_text="Date", row=3, col=1)
fig.update_yaxes(title_text="°C", row=1, col=1)
fig.update_yaxes(title_text="%", row=2, col=1)
fig.update_yaxes(title_text="$", row=3, col=1)

# Update the layout and add titles
fig.update_layout(
    height=900,  # Height in pixels
    width=1000,  # Width in pixels
    title_text="Temperature, Humidity, and Price Over Time with Action Indicators",
    showlegend=True
)

# Show the plot
fig.show()


In [37]:
# Create subplots with 2 y-axes on the left and 1 on the right
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Mean Temperature Trace
fig.add_trace(
    go.Scatter(x=data['date'], y=data['meantemp'], name='Mean Temperature (°C)', marker=dict(color='red')),
    secondary_y=False,
)

# Humidity Trace
fig.add_trace(
    go.Scatter(x=data['date'], y=data['humidity'], name='Humidity (%)', marker=dict(color='blue')),
    secondary_y=True,
)

# Price Trace with Actions
fig.add_trace(
    go.Scatter(x=data['date'], y=data['price'], name='Price ($)', mode='lines+markers',
               marker=dict(color=[{'Turn On': 'green', 'Turn Off': 'red'}[action] for action in data['action']])),
    secondary_y=True,
)

# Set x-axis title
fig.update_xaxes(title_text="Date")

# Set y-axes titles
fig.update_yaxes(title_text="Mean Temperature (°C) / Humidity (%)", secondary_y=False)
fig.update_yaxes(title_text="Price ($)", secondary_y=True)

# Add figure title and layout adjustments
fig.update_layout(
    title_text="Temperature, Humidity, and Price Over Time with Action Indicators",
    legend_title="Legend",
    height=600,
    width=1000
)

# Show plot
fig.show()


# Part 2 -> Intra day weather conditions to keep the cost consumption below the threshold

## Data generation

In [38]:
data_path = "/kaggle/input/madrid-weather-dataset-by-hours-20192022/weather_madrid_2019-2022.csv"
df = spark.read.csv(data_path, header=True, inferSchema=True)
df.show()

+---+-------------------+-----------+----------+--------------+--------+-------------------+---------------+-------------+
|_c0|               time|temperature|wind_speed|wind_direction|humidity|barometric_pressure|solar_radiation|precipitation|
+---+-------------------+-----------+----------+--------------+--------+-------------------+---------------+-------------+
|  0|2019-01-01 00:59:59|        3.1|      0.65|          64.0|    50.0|              949.0|            0.0|          0.0|
|  1|2019-01-01 01:59:59|        2.9|      0.95|          32.0|    49.0|              949.0|            0.0|          0.0|
|  2|2019-01-01 02:59:59|        3.1|      0.87|          62.0|    46.0|              949.0|            0.0|          0.0|
|  3|2019-01-01 03:59:59|        1.9|      0.73|          75.0|    51.0|              949.0|            0.0|          0.0|
|  4|2019-01-01 04:59:59|        0.7|      0.57|          53.0|    56.0|              949.0|            0.0|          0.0|
|  5|2019-01-01 

24/10/26 09:48:36 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , time, temperature, wind_speed, wind_direction, humidity, barometric_pressure, solar_radiation, precipitation
 Schema: _c0, time, temperature, wind_speed, wind_direction, humidity, barometric_pressure, solar_radiation, precipitation
Expected: _c0 but found: 
CSV file: file:///kaggle/input/madrid-weather-dataset-by-hours-20192022/weather_madrid_2019-2022.csv


In [39]:
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- time: timestamp (nullable = true)
 |-- temperature: double (nullable = true)
 |-- wind_speed: double (nullable = true)
 |-- wind_direction: double (nullable = true)
 |-- humidity: double (nullable = true)
 |-- barometric_pressure: double (nullable = true)
 |-- solar_radiation: double (nullable = true)
 |-- precipitation: double (nullable = true)



In [40]:
df = df.drop('wind_speed','wind_direction','solar_radiation','precipitation', '_c0','barometric_pressure')
df.show()

+-------------------+-----------+--------+
|               time|temperature|humidity|
+-------------------+-----------+--------+
|2019-01-01 00:59:59|        3.1|    50.0|
|2019-01-01 01:59:59|        2.9|    49.0|
|2019-01-01 02:59:59|        3.1|    46.0|
|2019-01-01 03:59:59|        1.9|    51.0|
|2019-01-01 04:59:59|        0.7|    56.0|
|2019-01-01 05:59:59|        0.5|    53.0|
|2019-01-01 06:59:59|        0.0|    55.0|
|2019-01-01 07:59:59|       -0.8|    60.0|
|2019-01-01 08:59:59|        0.0|    56.0|
|2019-01-01 09:59:59|        2.2|    48.0|
|2019-01-01 10:59:59|        3.1|    47.0|
|2019-01-01 11:59:59|        5.5|    39.0|
|2019-01-01 12:59:59|        7.8|    34.0|
|2019-01-01 13:59:59|       10.1|    31.0|
|2019-01-01 14:59:59|       10.4|    33.0|
|2019-01-01 15:59:59|       11.9|    30.0|
|2019-01-01 16:59:59|       10.1|    35.0|
|2019-01-01 17:59:59|        8.5|    38.0|
|2019-01-01 18:59:59|        7.7|    39.0|
|2019-01-01 19:59:59|        7.1|    43.0|
+----------

In [41]:
df.printSchema()

root
 |-- time: timestamp (nullable = true)
 |-- temperature: double (nullable = true)
 |-- humidity: double (nullable = true)



In [42]:
df = df.withColumn("DayOfYear", dayofyear(col("time")))
df = df.withColumn("Hour", hour(col("time")))

In [43]:
input_features = ['temperature', 'humidity', 'DayOfYear', 'Hour']
assembler = VectorAssembler(inputCols=input_features, outputCol="features")
df = assembler.transform(df)

In [44]:
df.show()

+-------------------+-----------+--------+---------+----+--------------------+
|               time|temperature|humidity|DayOfYear|Hour|            features|
+-------------------+-----------+--------+---------+----+--------------------+
|2019-01-01 00:59:59|        3.1|    50.0|        1|   0|  [3.1,50.0,1.0,0.0]|
|2019-01-01 01:59:59|        2.9|    49.0|        1|   1|  [2.9,49.0,1.0,1.0]|
|2019-01-01 02:59:59|        3.1|    46.0|        1|   2|  [3.1,46.0,1.0,2.0]|
|2019-01-01 03:59:59|        1.9|    51.0|        1|   3|  [1.9,51.0,1.0,3.0]|
|2019-01-01 04:59:59|        0.7|    56.0|        1|   4|  [0.7,56.0,1.0,4.0]|
|2019-01-01 05:59:59|        0.5|    53.0|        1|   5|  [0.5,53.0,1.0,5.0]|
|2019-01-01 06:59:59|        0.0|    55.0|        1|   6|  [0.0,55.0,1.0,6.0]|
|2019-01-01 07:59:59|       -0.8|    60.0|        1|   7| [-0.8,60.0,1.0,7.0]|
|2019-01-01 08:59:59|        0.0|    56.0|        1|   8|  [0.0,56.0,1.0,8.0]|
|2019-01-01 09:59:59|        2.2|    48.0|        1|

## Data Visualisation -> using plotly library to plot time series and other graphs

In [45]:
data = df.toPandas()
data

,time,temperature,humidity,DayOfYear,Hour,features
0,2019-01-01 00:59:59,3.1,50.0,1,0,"[3.1, 50.0, 1.0, 0.0]"
1,2019-01-01 01:59:59,2.9,49.0,1,1,"[2.9, 49.0, 1.0, 1.0]"
2,2019-01-01 02:59:59,3.1,46.0,1,2,"[3.1, 46.0, 1.0, 2.0]"
3,2019-01-01 03:59:59,1.9,51.0,1,3,"[1.9, 51.0, 1.0, 3.0]"
4,2019-01-01 04:59:59,0.7,56.0,1,4,"[0.7, 56.0, 1.0, 4.0]"
...,...,...,...,...,...,...
27019,2022-01-31 19:59:59,10.1,17.0,31,19,"[10.1, 17.0, 31.0, 19.0]"
27020,2022-01-31 20:59:59,8.2,25.0,31,20,"[8.2, 25.0, 31.0, 20.0]"
27021,2022-01-31 21:59:59,7.2,31.0,31,21,"[7.2, 31.0, 31.0, 21.0]"
27022,2022-01-31 22:59:59,5.6,41.0,31,22,"[5.6, 41.0, 31.0, 22.0]"


In [46]:
# Create subplots
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.1, subplot_titles=("Temperature Over Time", "Humidity Over Time"))

# Add temperature trace
fig.add_trace(
    go.Scatter(x=data['time'], y=data['temperature'], name='Temperature'),
    row=1, col=1
)

# Add humidity trace
fig.add_trace(
    go.Scatter(x=data['time'], y=data['humidity'], name='Humidity'),
    row=2, col=1
)

# Update layout
fig.update_layout(height=600, width=800, title_text="Temperature and Humidity Over Time")
fig.update_xaxes(title_text="Time", row=2, col=1)
fig.update_yaxes(title_text="Temperature (°C)", row=1, col=1)
fig.update_yaxes(title_text="Humidity (%)", row=2, col=1)

# Show plot
fig.show()

In [47]:
# Create subplots for area charts
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.1, subplot_titles=("Temperature Over Time", "Humidity Over Time"))

# Add temperature area chart
fig.add_trace(
    go.Scatter(x=data['time'], y=data['temperature'], name='Temperature', fill='tozeroy'),
    row=1, col=1
)

# Add humidity area chart
fig.add_trace(
    go.Scatter(x=data['time'], y=data['humidity'], name='Humidity', fill='tozeroy'),
    row=2, col=1
)

# Update layout
fig.update_layout(height=600, width=800, title_text="Temperature and Humidity Over Time")
fig.update_xaxes(title_text="Time", row=2, col=1)
fig.update_yaxes(title_text="Temperature (°C)", row=1, col=1)
fig.update_yaxes(title_text="Humidity (%)", row=2, col=1)

# Show plot
fig.show()

## Observations from the above plots
1. Both the temperature and humidity exhibit clear seasonal cycles. 
2. Temperature peaks during the middle of the year and reaches its lowest points toward the beginning and end of the year, indicating typical summer and winter seasons respectively.
3. Humidity displays somewhat inverse pattern to temperature, where it tends to be lower during hot months and higher during cooler months
4. Systems need to adapt to the varying changes of tempr and humidity and thus efficiently adjust the heating and cooling systems in a cost effective way.

## Defining the model

### Since I am taking dataset of Madrid, here the cost for the appliance heaters
The following code is effectively managing energy use based on external climate conditions and internal temperature settings. Cumulative cost calculation and temperature optimization to keep the cost below a particular threshold for each day.

In [48]:
from pyspark.sql.types import DoubleType
import pyspark.sql.functions as F

# Function to simulate cost based on room and external temperatures
def calculate_cost(room_temp, outside_temp, humidity):
    # The coefficient increases as the difference between room and outside temperature increases
    temp_diff = room_temp - outside_temp
    return (0.05 * temp_diff ** 2) + (0.03 * humidity)

# Register UDF
calculate_cost_udf = udf(calculate_cost, DoubleType())

# Assuming initial room temperature
initial_room_temperature = 22.0

# Add a column for simulated costs
df = df.withColumn("predicted_cost", calculate_cost_udf(F.lit(initial_room_temperature), col("temperature"), col("humidity")))

# Define a threshold for daily costs
daily_threshold = 10.0  # Adjust as necessary

# Window specification for cumulative cost
windowSpec = Window.partitionBy("DayOfYear").orderBy("time").rowsBetween(Window.unboundedPreceding, 0)

# Calculate cumulative cost for each hour
df = df.withColumn("cumulative_cost", F.sum("predicted_cost").over(windowSpec))

# Optimization logic to find optimal room temperature
def optimize_temperature(cumulative_cost, room_temp, outside_temp, step=0.1, max_iter=100):
    iteration = 0
    while cumulative_cost > daily_threshold and iteration < max_iter:
        room_temp -= step  # Adjust temperature
        # Recalculate cost with new temperature
        cumulative_cost -= calculate_cost(room_temp, outside_temp, 50)  # Assuming constant humidity for simplification
        iteration += 1
    return room_temp

# Register the optimization UDF
optimize_temperature_udf = udf(optimize_temperature, DoubleType())

# Apply the UDF to adjust the room temperature
df = df.withColumn("optimal_temperature", optimize_temperature_udf(col("cumulative_cost"), F.lit(initial_room_temperature), col("temperature")))

# Show the results
df.select("DayOfYear", "time", "temperature", "humidity", "predicted_cost", "cumulative_cost", "optimal_temperature").show()


+---------+-------------------+-----------+--------+------------------+------------------+-------------------+
|DayOfYear|               time|temperature|humidity|    predicted_cost|   cumulative_cost|optimal_temperature|
+---------+-------------------+-----------+--------+------------------+------------------+-------------------+
|        1|2019-01-01 00:59:59|        3.1|    50.0|           19.3605|           19.3605|               21.9|
|        1|2019-01-01 01:59:59|        2.9|    49.0|19.710500000000003|            39.071| 21.799999999999997|
|        1|2019-01-01 02:59:59|        3.1|    46.0|19.240499999999997|58.311499999999995| 21.699999999999996|
|        1|2019-01-01 03:59:59|        1.9|    51.0|21.730500000000006|            80.042| 21.599999999999994|
|        1|2019-01-01 04:59:59|        0.7|    56.0|24.364500000000003|104.40650000000001| 21.599999999999994|
|        1|2019-01-01 05:59:59|        0.5|    53.0|           24.7025|           129.109| 21.499999999999993|
|

### Trying with a different threshold cost of 2 dollars each day

In [49]:
# Define a UDF to simulate the cost based on temperature and humidity
def simulate_cost(temp, humidity):
    # Placeholder for actual cost calculation logic
    return temp * 0.04 + humidity * 0.02

# Register the UDF
simulate_cost_udf = udf(simulate_cost, DoubleType())

# Apply the cost simulation UDF across your data
df = df.withColumn("predicted_cost", simulate_cost_udf(col("temperature"), col("humidity")))

# Set a daily threshold for costs
daily_threshold = 2  # This could be dynamic and linked to external data sources or conditions

# Define the window specification for cumulative cost calculation
# This will reset the cumulative cost at the start of each new day
windowSpec = Window.partitionBy("DayOfYear").orderBy("Hour").rowsBetween(Window.unboundedPreceding, 0)

# Calculate cumulative cost for each hour, resetting for each new day
df = df.withColumn("cumulative_cost", _sum("predicted_cost").over(windowSpec))

# Define the optimization logic to adjust the temperature
def find_optimal_temperature(cumulative_cost, current_temp, step=0.5, max_iter=20):
    iteration = 0
    # Adjust the temperature to reduce the cost
    while cumulative_cost > daily_threshold and iteration < max_iter:
        current_temp -= step  # Decrease temperature to try reducing cost
        cumulative_cost -= step * 0.04  # Direct cost reduction for simplicity
        iteration += 1
    return current_temp

# Register the optimization UDF
find_optimal_temperature_udf = udf(find_optimal_temperature, DoubleType())

# Apply the UDF to determine optimal temperature adjustments
df = df.withColumn("optimal_temperature", find_optimal_temperature_udf(col("cumulative_cost"), col("temperature")))

# Show the results
df.show()


+-------------------+-----------+--------+---------+----+------------------+------------------+------------------+-------------------+
|               time|temperature|humidity|DayOfYear|Hour|          features|    predicted_cost|   cumulative_cost|optimal_temperature|
+-------------------+-----------+--------+---------+----+------------------+------------------+------------------+-------------------+
|2019-01-01 00:59:59|        3.1|    50.0|        1|   0|[3.1,50.0,1.0,0.0]|             1.124|             1.124|                3.1|
|2020-01-01 00:59:59|        2.8|    82.0|        1|   0|[2.8,82.0,1.0,0.0]|1.7520000000000002|2.8760000000000003|               -7.2|
|2021-01-01 00:59:59|        0.6|    89.0|        1|   0|[0.6,89.0,1.0,0.0]|             1.804| 4.680000000000001|               -9.4|
|2022-01-01 00:59:59|        6.9|    71.0|        1|   0|[6.9,71.0,1.0,0.0]|             1.696|             6.376|-3.0999999999999996|
|2019-01-01 01:59:59|        2.9|    49.0|        1|   

In [50]:
df.printSchema()

root
 |-- time: timestamp (nullable = true)
 |-- temperature: double (nullable = true)
 |-- humidity: double (nullable = true)
 |-- DayOfYear: integer (nullable = true)
 |-- Hour: integer (nullable = true)
 |-- features: vector (nullable = true)
 |-- predicted_cost: double (nullable = true)
 |-- cumulative_cost: double (nullable = true)
 |-- optimal_temperature: double (nullable = true)



In [52]:
new_df = df

#### The code below is now simply creating another column for updated predicted price based on the temperature difference between optimal temperature that is found above and external temperature, the updated predicted price is the price that the user is paying for that hour and updated cumulative price is the one that the user is paying daily

In [53]:

# Define the UDF to recalculate cost based on the adjusted optimal temperature
def recalculate_cost(opt_temp, outside_temp, humidity):
    # Simulate the recalculation of cost based on the new optimal temperature
    # Modify this function according to your specific cost calculation logic
    temp_diff = opt_temp - outside_temp
    return (0.05 * temp_diff ** 2) + (0.03 * humidity)

recalculate_cost_udf = udf(recalculate_cost, DoubleType())

# Recalculate predicted cost using the optimal temperature
new_df = new_df.withColumn("updated_predicted_cost", recalculate_cost_udf(col("optimal_temperature"),col("temperature"), col("humidity")))

# Update the cumulative cost to reflect the cost with the new temperature
# Redefine the window specification for cumulative cost calculation
windowSpec = Window.partitionBy("DayOfYear").orderBy("time").rowsBetween(Window.unboundedPreceding, 0)

# Calculate the updated cumulative cost for each hour using the newly predicted cost
new_df = new_df.withColumn("updated_cumulative_cost", _sum("updated_predicted_cost").over(windowSpec))

# Add logic to check and update costs only if the new cumulative cost is less than the threshold
new_df = new_df.withColumn("final_cumulative_cost",
                   when(col("updated_cumulative_cost") <= daily_threshold, col("updated_cumulative_cost"))
                   .otherwise(col("cumulative_cost")))


In [54]:
new_df.printSchema()

root
 |-- time: timestamp (nullable = true)
 |-- temperature: double (nullable = true)
 |-- humidity: double (nullable = true)
 |-- DayOfYear: integer (nullable = true)
 |-- Hour: integer (nullable = true)
 |-- features: vector (nullable = true)
 |-- predicted_cost: double (nullable = true)
 |-- cumulative_cost: double (nullable = true)
 |-- optimal_temperature: double (nullable = true)
 |-- updated_predicted_cost: double (nullable = true)
 |-- updated_cumulative_cost: double (nullable = true)
 |-- final_cumulative_cost: double (nullable = true)



In [56]:
new_df.show()

+-------------------+-----------+--------+---------+----+--------------------+------------------+------------------+-------------------+----------------------+-----------------------+---------------------+
|               time|temperature|humidity|DayOfYear|Hour|            features|    predicted_cost|   cumulative_cost|optimal_temperature|updated_predicted_cost|updated_cumulative_cost|final_cumulative_cost|
+-------------------+-----------+--------+---------+----+--------------------+------------------+------------------+-------------------+----------------------+-----------------------+---------------------+
|2019-01-01 00:59:59|        3.1|    50.0|        1|   0|  [3.1,50.0,1.0,0.0]|             1.124|             1.124|                3.1|                   1.5|                    1.5|                  1.5|
|2019-01-01 01:59:59|        2.9|    49.0|        1|   1|  [2.9,49.0,1.0,1.0]|             1.096|             7.472|               -7.1|                  6.47|                 

In [57]:
new_df.select("updated_predicted_cost").show()

+----------------------+
|updated_predicted_cost|
+----------------------+
|                   1.5|
|                  7.46|
|                  7.67|
|                  7.13|
|                  6.47|
|                  7.52|
|                  7.52|
|    7.1899999999999995|
|                  6.38|
|                  7.52|
|                  7.37|
|                  7.25|
|                  6.53|
|                  7.49|
|                  7.43|
|                  7.13|
|                  6.68|
|                  7.49|
|                  7.43|
|                  6.98|
+----------------------+
only showing top 20 rows



In [58]:
new_df.select("updated_cumulative_cost").show()

+-----------------------+
|updated_cumulative_cost|
+-----------------------+
|                    1.5|
|                   7.97|
|                  14.35|
|                  20.88|
|                  27.56|
|                  34.15|
|                   40.8|
|     47.599999999999994|
|     54.279999999999994|
|      60.71999999999999|
|                  67.13|
|                   73.3|
|                  79.32|
|                  85.25|
|                  91.24|
|                  97.14|
|                 103.19|
|                 109.33|
|                  115.5|
|                 121.79|
+-----------------------+
only showing top 20 rows

